In [1]:
import pandas as pd
import os
import pickle

import networkx as nx

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from src.dataset.dataset_info import datasets
from src.graph.graph_measures import calculate_graph_measures
from src.graph.graph_construction.window_graph import create_weightless_window_graph 

In [2]:
flow_graph = False
session_graph = False

window_graph = True
window_size=2000

multi_graph=False
line_graph = True

In [3]:
name = "cic_ids_2017_5_percent"
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
# name = "cic_ids_2017"
# name = "nf_bot_iot"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"

dataset = datasets[name]

In [4]:
df = pd.read_parquet(dataset.path)

In [5]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
index,,,,,,,,,,,,,,,,,,,,,
66292,192.168.10.1-192.168.10.3-53-60671-17,192.168.10.3,60671.0,192.168.10.1,53.0,17.0,03/07/2017 10:23:37,30919.0,1.0,1.0,...,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
420414,192.168.10.8-52.84.64.212-51938-443-6,52.84.64.212,443.0,192.168.10.8,51938.0,6.0,03/07/2017 11:20:18,3.0,1.0,1.0,...,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
393869,172.217.10.2-192.168.10.8-80-55154-6,192.168.10.8,55154.0,172.217.10.2,80.0,6.0,5/7/2017 1:53,115525809.0,16.0,14.0,...,140027.3781,501173.0,36578.0,9976579.636,58066.24315,9994305.0,9801504.0,0,BENIGN,0
319307,192.168.10.1-192.168.10.3-53-61248-17,192.168.10.3,61248.0,192.168.10.1,53.0,17.0,03/07/2017 04:15:49,60594.0,1.0,1.0,...,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
419002,192.168.10.3-192.168.10.12-53-32968-17,192.168.10.12,32968.0,192.168.10.3,53.0,17.0,03/07/2017 09:40:12,310.0,2.0,2.0,...,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0


In [6]:
cols_to_norm = list(set(list(df.columns ))  - set(list([dataset.label_col]))  - set(list([dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))
cols_to_norm

['Bwd Header Len',
 'CWE Flag Count',
 'Active Min',
 'Bwd IAT Min',
 'PSH Flag Cnt',
 'Bwd IAT Max',
 'Subflow Fwd Byts',
 'Fwd Pkt Len Min',
 'Idle Min',
 'Pkt Len Min',
 'Fwd Pkts/s',
 'SYN Flag Cnt',
 'URG Flag Cnt',
 'Bwd Pkts/s',
 'FIN Flag Cnt',
 'Fwd Seg Size Min',
 'Bwd IAT Tot',
 'Down/Up Ratio',
 'Idle Std',
 'Fwd Seg Size Avg',
 'ACK Flag Cnt',
 'Flow Byts/s',
 'Bwd Pkt Len Min',
 'Bwd Seg Size Avg',
 'ECE Flag Cnt',
 'Active Max',
 'Fwd Header Len',
 'Fwd Act Data Pkts',
 'Init Fwd Win Byts',
 'Flow IAT Min',
 'Init Bwd Win Byts']

In [7]:
df.Label.value_counts()

Label
0    113501
1     27754
Name: count, dtype: int64

In [8]:
if window_graph and line_graph:
    df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.reset_index(drop=False, inplace=True)
    
    scaler = StandardScaler()

    df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])
    df['h'] = df[ cols_to_norm ].values.tolist()
    
    create_weightless_window_graph(
        df=df,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=True,
        folder_path="datasets/" + name + "/line_graph",
        # folder_path=None,
        test_percentage = 20,
        edge_attr= ['h',dataset.label_col, "index"],
        file_type="pkl")
    
    

==>> number_of_groups: 71
==>> number_of_train_groups: 56
==>> number_of_test_groups: 15
==>> graph_measures: {'number_of_nodes': 2000, 'number_of_edges': 240721, 'max_degree': 884, 'avg_degree': 240.721, 'density': 0.06021035517758879, 'number_of_communities': 112, 'mixing_parameter': 0.1649793744625521, 'modularity': 0.023925227637240344}
==>> graph_measures: {'number_of_nodes': 2000, 'number_of_edges': 277290, 'max_degree': 766, 'avg_degree': 277.29, 'density': 0.06935717858929465, 'number_of_communities': 74, 'mixing_parameter': 0.07573298712539218, 'modularity': 0.009428618972344814}
==>> graph_measures: {'number_of_nodes': 2000, 'number_of_edges': 264815, 'max_degree': 844, 'avg_degree': 264.815, 'density': 0.06623686843421711, 'number_of_communities': 85, 'mixing_parameter': 0.10691992523082151, 'modularity': 0.01642600975544806}
==>> graph_measures: {'number_of_nodes': 2000, 'number_of_edges': 255385, 'max_degree': 942, 'avg_degree': 255.385, 'density': 0.06387818909454727, 'nu

In [9]:
if not window_graph:
        y = df[dataset.label_col]
        X_train, X_test, y_train, y_test = train_test_split(
                df, y, test_size=0.3, random_state=13, stratify=y)

        del df

        print(f"==>> X_train.shape: {X_train.shape}")
        print(f"==>> X_train.shape: {X_test.shape}")

        scaler = StandardScaler()

        X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
        X_train['h'] = X_train[ cols_to_norm ].values.tolist()

        X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
        X_test['h'] = X_test[ cols_to_norm ].values.tolist()

In [10]:
if flow_graph:
    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col], create_using=nx.MultiDiGraph())
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, "datasets/" + name + "/training_graph_measures.json")
    print(f"==>> graph_measures: {graph_measures}")
    
    graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json")
    print(f"==>> graph_measures: {graph_measures}")
    
    with open("datasets/" + name + "/training_graph.pkl", "wb") as f:
        pickle.dump(G, f)

In [11]:
if flow_graph:
    G_test = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col],create_using=nx.MultiDiGraph())
    
    graph_measures = calculate_graph_measures(G_test, "datasets/" + name + "/testing_graph_measures.json")
    print(f"==>> graph_measures: {graph_measures}")
    
    graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json")
    print(f"==>> graph_measures: {graph_measures}")
    
    with open("datasets/" + name + "/testing_graph.pkl", "wb") as f:
        pickle.dump(G_test, f)

In [12]:
# if window_graph and line_graph:

#     create_weightless_window_graph(
#         df=X_test,
#         src_ip_col=dataset.src_ip_col,
#         dst_ip_col=dataset.dst_ip_col,
#         window_size=window_size,
#         line_graph=True,
#         folder_path="datasets/" + name + "/line_graph/testing",
#         file_type="pkl")

In [13]:
# nx.write_gexf(G_test, "datasets/cic_ton_iot/testing_graph.gexf")